In [3]:
# Instalar las librerías necesarias
!pip install qrcode[pil] pillow

In [7]:
# importacion de librerias
import qrcode
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
import io
import os

In [ ]:
def subir_imagen():
    """
    Permite subir una imagen desde el equipo local en Colab
    """
    print("Por favor, sube una imagen para personalizar el QR:")
    uploaded = files.upload()

    if not uploaded:
        print("No se subió ninguna imagen.")
        return None

    # Obtener el nombre del archivo subido
    nombre_archivo = list(uploaded.keys())[0]
    print(f"Imagen subida: {nombre_archivo}")

    return nombre_archivo

def procesar_imagen_centro(ruta_imagen, tamaño_centro=100):
    """
    Procesa la imagen para colocarla en el centro del QR

    Args:
        ruta_imagen (str): Ruta de la imagen a procesar
        tamaño_centro (int): Tamaño en píxeles para la imagen del centro

    Returns:
        PIL.Image: Imagen procesada y redimensionada
    """
    try:
        # Abrir la imagen
        imagen_centro = Image.open(ruta_imagen)

        # Convertir a RGB si es necesario
        if imagen_centro.mode != 'RGB':
            imagen_centro = imagen_centro.convert('RGB')

        # Redimensionar manteniendo la relación de aspecto
        ancho, alto = imagen_centro.size
        tamaño_max = max(ancho, alto)

        # Crear una imagen cuadrada con fondo blanco
        imagen_cuadrada = Image.new('RGB', (tamaño_max, tamaño_max), 'white')

        # Pegar la imagen original en el centro
        offset_x = (tamaño_max - ancho) // 2
        offset_y = (tamaño_max - alto) // 2
        imagen_cuadrada.paste(imagen_centro, (offset_x, offset_y))

        # Redimensionar al tamaño final
        imagen_centro = imagen_cuadrada.resize((tamaño_centro, tamaño_centro), Image.LANCZOS)

        return imagen_centro

    except Exception as e:
        print(f"Error al procesar la imagen: {e}")
        return None

def generar_qr_con_imagen_centro(url, ruta_imagen_centro=None, imagen_centro=None,
                                tamaño_centro=100, nombre_archivo="qr_personalizado.png"):
    """
    Genera un código QR con una imagen en el centro

    Args:
        url (str): URL para el código QR
        ruta_imagen_centro (str): Ruta de la imagen para el centro
        imagen_centro (PIL.Image): Imagen ya cargada para el centro
        tamaño_centro (int): Tamaño de la imagen central en píxeles
        nombre_archivo (str): Nombre del archivo de salida

    Returns:
        PIL.Image: Código QR personalizado
    """

    # Crear el código QR básico
    qr = qrcode.QRCode(
        version=5,  # Versión más grande para espacio en el centro
        error_correction=qrcode.constants.ERROR_CORRECT_H,  # Alta corrección de errores
        box_size=10,
        border=4,
    )

    qr.add_data(url)
    qr.make(fit=True)

    # Crear imagen del QR
    img_qr = qr.make_image(fill_color="black", back_color="white").convert('RGB')

    # Procesar la imagen del centro si se proporciona una ruta
    if ruta_imagen_centro and not imagen_centro:
        imagen_centro = procesar_imagen_centro(ruta_imagen_centro, tamaño_centro)

    # Si tenemos una imagen para el centro, agregarla
    if imagen_centro:
        # Calcular posición para centrar la imagen
        pos_x = (img_qr.size[0] - imagen_centro.size[0]) // 2
        pos_y = (img_qr.size[1] - imagen_centro.size[1]) // 2

        # Pegar la imagen en el centro del QR
        img_qr.paste(imagen_centro, (pos_x, pos_y))

    return img_qr

def mostrar_y_guardar_qr(img_qr, url, nombre_archivo="qr_personalizado.png"):
    """
    Muestra y guarda el código QR generado
    """
    # Mostrar el QR
    plt.figure(figsize=(10, 10))
    plt.imshow(img_qr)
    plt.title(f"Código QR para: {url}")
    plt.axis('off')
    plt.show()

    # Guardar el archivo
    img_qr.save(nombre_archivo)
    print(f"✅ Código QR guardado como: {nombre_archivo}")

    # Ofrecer descarga
    print("¿Quieres descargar el código QR?")
    files.download(nombre_archivo)

    return img_qr

def ejemplo_qr_con_imagen():
    """
    Ejemplo de generación de QR con imagen personalizada
    """
    print("=== Generador de QR con Imagen Personalizada ===\n")

    # Solicitar URL
    url = input("Ingresa la URL para el código QR: ").strip()
    if not url.startswith(('http://', 'https://')):
        url = 'https://' + url

    # Opción: subir imagen o usar una por defecto
    print("\nOpciones para la imagen central:")
    print("1. Subir una imagen desde mi equipo")
    print("2. Usar imagen de ejemplo (logo)")
    print("3. Solo QR sin imagen central")

    opcion = input("Selecciona una opción (1-3): ").strip()

    imagen_centro = None
    ruta_imagen = None

    if opcion == "1":
        # Subir imagen
        ruta_imagen = subir_imagen()
        if not ruta_imagen:
            print("Usando QR sin imagen central.")

    elif opcion == "2":
        # Crear un logo simple de ejemplo
        try:
            # Crear una imagen simple de ejemplo
            tamaño = 100
            imagen_centro = Image.new('RGB', (tamaño, tamaño), 'blue')

            # Dibujar un círculo blanco
            from PIL import ImageDraw
            draw = ImageDraw.Draw(imagen_centro)
            draw.ellipse([10, 10, tamaño-10, tamaño-10], fill='white')

            # Agregar texto
            draw.text((tamaño//2-15, tamaño//2-10), "QR", fill='black')

            print("Usando imagen de ejemplo.")

        except Exception as e:
            print(f"Error creando imagen de ejemplo: {e}")

    # Generar el QR personalizado
    try:
        nombre_archivo = f"qr_personalizado_{url.split('//')[-1].replace('/', '_')}.png"
        qr_personalizado = generar_qr_con_imagen_centro(
            url=url,
            ruta_imagen_centro=ruta_imagen,
            imagen_centro=imagen_centro,
            tamaño_centro=100,
            nombre_archivo=nombre_archivo
        )

        # Mostrar y guardar
        mostrar_y_guardar_qr(qr_personalizado, url, nombre_archivo)

    except Exception as e:
        print(f"❌ Error al generar el QR: {e}")

def generar_varios_qr_con_imagen():
    """
    Genera múltiples códigos QR con la misma imagen central
    """
    print("=== Generador Múltiple de QR con Imagen ===\n")

    # Subir imagen una vez
    ruta_imagen = subir_imagen()
    if not ruta_imagen:
        return

    # Solicitar URLs
    print("\nIngresa las URLs (una por línea, línea vacía para terminar):")
    urls = []
    while True:
        url = input().strip()
        if not url:
            break
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url
        urls.append(url)

    if not urls:
        print("No se ingresaron URLs.")
        return

    # Generar QRs para cada URL
    for i, url in enumerate(urls, 1):
        try:
            nombre_archivo = f"qr_{i}_{url.split('//')[-1].replace('/', '_')}.png"
            qr_personalizado = generar_qr_con_imagen_centro(
                url=url,
                ruta_imagen_centro=ruta_imagen,
                tamaño_centro=80
            )

            qr_personalizado.save(nombre_archivo)
            print(f"✅ QR {i} guardado: {nombre_archivo}")

            # Mostrar el primero como ejemplo
            if i == 1:
                plt.figure(figsize=(8, 8))
                plt.imshow(qr_personalizado)
                plt.title(f"Ejemplo: QR para {url}")
                plt.axis('off')
                plt.show()

        except Exception as e:
            print(f"❌ Error con QR {i}: {e}")

# Función principal
def main():
    """
    Menú principal del generador de QR
    """
    while True:
        print("\n" + "="*50)
        print("        GENERADOR DE CÓDIGOS QR PERSONALIZADOS")
        print("="*50)
        print("1. Generar un QR con imagen personalizada")
        print("2. Generar múltiples QRs con la misma imagen")
        print("3. Generar QR simple (sin imagen)")
        print("4. Salir")

        opcion = input("\nSelecciona una opción (1-4): ").strip()

        if opcion == "1":
            ejemplo_qr_con_imagen()
        elif opcion == "2":
            generar_varios_qr_con_imagen()
        elif opcion == "3":
            # Usar tu función original
            from google.colab import auth
            auth.authenticate_user()

            url = input("Ingresa la URL: ").strip()
            if not url.startswith(('http://', 'https://')):
                url = 'https://' + url

            generar_qr_desde_url(url, mostrar=True, guardar=True)
        elif opcion == "4":
            print("¡Hasta luego!")
            break
        else:
            print("Opción no válida. Por favor, selecciona 1-4.")

# Ejecutar el programa
if __name__ == "__main__":
    # Asegurarse de que tu función original esté disponible
    def generar_qr_desde_url(url, nombre_archivo="codigo_qr.png", mostrar=True, guardar=True):
        qr = qrcode.QRCode(
            version=1,
            error_correction=qrcode.constants.ERROR_CORRECT_L,
            box_size=10,
            border=4,
        )
        qr.add_data(url)
        qr.make(fit=True)
        img_qr = qr.make_image(fill_color="black", back_color="white")

        if mostrar:
            plt.figure(figsize=(8, 8))
            plt.imshow(img_qr, cmap='gray')
            plt.title(f"QR para: {url}")
            plt.axis('off')
            plt.show()

        if guardar:
            img_qr.save(nombre_archivo)
            print(f"QR guardado como: {nombre_archivo}")

        return img_qr

    # Ejecutar el menú principal
    main()